In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import kpss


from google.colab import files
uploaded = files.upload()

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Saving 2015_suicide_no.csv to 2015_suicide_no.csv
Saving test_mini1.csv to test_mini1.csv
Saving train1.csv to train1.csv


In [0]:
df = pd.read_csv('./train1.csv')
df_test = pd.read_csv('./test_mini1.csv')


#df=df['year'].astype(int)
df.head()

,country,year,sex,age,suicides_no,population,suicides_100k_pop,country_year,HDI_for_year,gdp_for_year,gdp_per_capital,generation,Social support,Healthy life expectancy at birth,Positive affect,Negative affect
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,0.635,2.156625e+09,796,Generation X,NaN,NaN,NaN,NaN
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,0.635,2.156625e+09,796,Silent,NaN,NaN,NaN,NaN
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,0.635,2.156625e+09,796,Generation X,NaN,NaN,NaN,NaN
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,0.635,2.156625e+09,796,G.I. Generation,NaN,NaN,NaN,NaN
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,0.635,2.156625e+09,796,Boomers,NaN,NaN,NaN,NaN


In [0]:
from statsmodels.tsa.stattools import adfuller

def Dickey_Fuller_Test(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=1).mean()
    rolstd = timeseries.rolling(window=1).std()

    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)


In [0]:


def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)


In [0]:

 
# evaluate an ARIMA model for a given order (p,d,q) and return RMSE
def evaluate_arima_model(X, arima_order):
# prepare training dataset
	X = X.astype('float32')
	train_size = int(len(X) * 0.70)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		# model_fit = model.fit(disp=0)
		model_fit = model.fit(trend='nc', disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	mse = mean_squared_error(test, predictions)
	rmse = sqrt(mse)
	return rmse


# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s RMSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 



In [0]:

from pandas import Series, DataFrame

df_predict = DataFrame(columns=('country', 'suicides_no_predicted'))
  

df['country'].unique()[0]

for i in df['country'].unique():
  df_rf=df[df['country']==i]
  

  
  suicides=df_rf[['year','suicides_no']].copy()

  suicides['year']=suicides['year'].astype(str)
  suicides['year']=pd.to_datetime(suicides['year'],format='%Y')

  suicides = suicides.groupby('year')['suicides_no'].sum().reset_index()


  suicides = suicides.set_index('year')


  suicides = suicides['suicides_no']

  if i== 'Albania' :
    Dickey_Fuller_Test(suicides)
    kpss_test(suicides)
  
  series = suicides.copy()
  # evaluate parameters
  p_values = range(0, 4)
  d_values = range(0, 2)
  q_values = range(0, 5)
  warnings.filterwarnings("ignore")
  evaluate_models(series, p_values, d_values, q_values)



  # Predicting the next five years suicides number
  history = suicides.copy()
  history=list(history.values)
  predictions = list()
  
  
  y=2015

  try:
    model = ARIMA(history, order=(2,0,0))
    model_fit = model.fit(trend='nc', disp=0)
    yhat = model_fit.forecast()[0]
                              
    print('\n For Country ',i,' For year',y, 'forecasted suicide number is',yhat,'\n')
    df_predict=df_predict.append({'country':i, 'suicides_no_predicted':yhat},ignore_index=True)

  except Exception as e : 
    #df_predict=df_predict.append({'country':i, 'suicides_no_predicted':'null' },ignore_index=True   )
    print(e)
    pass
  
df_predict

    


Results of Dickey-Fuller Test:
Test Statistic                 -0.619734
p-value                         0.866460
#Lags Used                      8.000000
Number of Observations Used    13.000000
Critical Value (1%)            -4.068854
Critical Value (5%)            -3.127149
Critical Value (10%)           -2.701730
dtype: float64
Results of KPSS Test:
Test Statistic           0.171043
p-value                  0.100000
Lags Used                9.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64
ARIMA(0, 0, 1) RMSE=88.752
ARIMA(0, 0, 2) RMSE=125.335
ARIMA(0, 0, 3) RMSE=125.929
ARIMA(0, 1, 1) RMSE=89.218
ARIMA(0, 1, 2) RMSE=78.716
ARIMA(1, 0, 0) RMSE=97.664
ARIMA(1, 1, 0) RMSE=101.197
ARIMA(2, 0, 0) RMSE=98.848
ARIMA(2, 1, 0) RMSE=96.897
ARIMA(3, 0, 0) RMSE=97.186
ARIMA(3, 1, 0) RMSE=94.813
Best ARIMA(0, 1, 2) RMSE=78.716

 For Country  Albania  For year 2015 forecasted suicide num

,country,suicides_no_predicted
0,Albania,57.540571
1,Antigua and Barbuda,0.000000
2,Argentina,3201.424467
3,Armenia,58.688424
4,Aruba,7.601216
5,Australia,2900.391631
6,Austria,1312.810793
7,Azerbaijan,63.752064
8,Bahamas,4.051212
9,Bahrain,8.375136


In [0]:

from google.colab import files
uploaded = files.upload()



answer= pd.read_csv('./2015_suicide_no.csv')



merge_answer= pd.merge(df_predict, answer , on ='country')

merge_answer['suicides_no_predicted']=merge_answer['suicides_no_predicted'].astype(float)

merge_answer['error']=merge_answer['suicides_no']-merge_answer['suicides_no_predicted']
merge_answer['error_Abs'] = abs(merge_answer['error'])
merge_answer      

merge_answer=merge_answer.dropna()
merge_answer

,country,suicides_no_predicted,suicides_no,error,error_Abs
0,Antigua and Barbuda,0.000000,1,1.000000,1.000000
1,Argentina,3201.424467,3073,-128.424467,128.424467
2,Armenia,58.688424,74,15.311576,15.311576
3,Australia,2900.391631,3027,126.608369,126.608369
4,Austria,1312.810793,1251,-61.810793,61.810793
5,Belgium,1895.876252,1867,-28.876252,28.876252
6,Belize,20.559775,26,5.440225,5.440225
7,Brazil,10666.258162,11163,496.741838,496.741838
8,Chile,1790.287032,1838,47.712968,47.712968
9,Colombia,2122.459316,2332,209.540684,209.540684


In [0]:

y_true = merge_answer.suicides_no
y_pred = merge_answer.suicides_no_predicted


print(np.sqrt(mean_squared_error(y_true, y_pred)))

197.0929657670189
